# Learning Chemical Classification Programs

This uses LLMs to learn programs for classifying chemical structures (SMILES strings) into chemical classes or groupings

## Data Model

We will set up a small pydantic data model for

- `ChemicalStructure` - something with a SMILES, typically "leafy"
- `ChemicalClass` - no SMILES, but groups terms with a SMILES

In [1]:
import random

from pydantic import BaseModel, Field
from typing import List, Optional


class ChemicalStructure(BaseModel):
    """Represents a chemical entity with a known specific structure/formula."""
    name: str = Field(..., description="rdfs:label of the structure in CHEBI")
    smiles: str = Field(..., description="SMILES string derived from CHEBI")

In [2]:
class ChemicalClass(BaseModel):
    """Represents a class/grouping of chemical entities."""
    id: str = Field(..., description="id/curie of the CHEBI class")
    name: str = Field(..., description="rdfs:label of the class in CHEBI")
    definition: str = Field(..., description="definition of the structure from CHEBI")
    instances: List[ChemicalStructure] = Field(..., description="positive examples")
    negative_instances: Optional[List[ChemicalStructure]] = Field(default=None, description="negative examples")
    

## LLM Setup 

We will use the datasette LLM library.

We will generate system prompts using example programs, and the main prompt paramaterized by the chemical class

In [3]:
import llm
from llm import Model, get_model

In [4]:
##

In [5]:
example_dir = "inputs"
validated_examples = ["benzenoid"]


In [6]:


def generate_system_prompt(examples: List[str]):
    system_prompt = """
    Write a program to classify chemical entities of a given class based on their SMILES string.
    
    The program should consist of import statements (from rdkit) as well as a single function, named
    is_<<chemical_class>> that takes a SMILES string as input and returns a boolean value plus a reason for the classification.
    
    If the task is too hard or cannot be done, the program MAY return None, None.
    
    You should ONLY include python code in your response. Do not include any markdown or other text, or
    non-code separators.
    If you wish to include explanatory information, then you can include them as code comments.
    
    """
    for example in examples:
        system_prompt += f"Here is an example for the chemical class {example}:\n{example}.py\n---\n"
        with open(f"{example_dir}/{example}.py", "r") as f:
            system_prompt += f.read()
    return system_prompt


In [7]:
print(generate_system_prompt(validated_examples))


    Write a program to classify chemical entities of a given class based on their SMILES string.
    
    The program should consist of import statements (from rdkit) as well as a single function, named
    is_<<chemical_class>> that takes a SMILES string as input and returns a boolean value plus a reason for the classification.
    
    If the task is too hard or cannot be done, the program MAY return None, None.
    
    You should ONLY include python code in your response. Do not include any markdown or other text, or
    non-code separators.
    If you wish to include explanatory information, then you can include them as code comments.
    
    Here is an example for the chemical class benzenoid:
benzenoid.py
---
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.Chem import rdMolDescriptors


def is_benzenoid(smiles: str):
    """
    Determines if a molecule is a benzenoid (benzene or substituted benzene).

    Args:
        smile

In [8]:
def safe_name(name: str) -> str:
    return "".join([c if c.isalnum() else "_" for c in name])

safe_name("foo' 3<->x bar")

'foo__3___x_bar'

In [9]:


def generate_main_prompt(chemical_class: str, definition: str, instances: List[ChemicalStructure], err=None, prog=None):
    # replace all non-alphanumeric characters with underscores
    chemical_class_safe = safe_name(chemical_class)
    prompt = f"""
    Now create a program that classifies chemical entities of the class {chemical_class},
    defined as '{definition}'. The name of the function should be `is_{chemical_class_safe}`.
    Examples of structures that belong to this class are:
    """
    for instance in instances:
        prompt += f" - {instance.name}: SMILES: {instance.smiles}\n"
    if err:
        prompt += f"\nYour last attempt failed with the following error: {err}\n"
        if prog:
            prompt += f"\nYour last attempt was:\n{prog}\n"
    return prompt

## Running generated code

We will need the rdkit library, which is imported in generated code

In [10]:
from typing import Tuple


def run_code(code_str: str, function_name: str, args: List[str]) -> List[Tuple[str, bool, str]]:
    exec(code_str, globals())
    vals = []
    for arg in args:
        func_exec_str = f"{function_name}('{arg}')"
        # print(f"Running: {func_exec_str}")
        r = eval(func_exec_str)
        vals.append((arg, *r))
    return vals


In [11]:
import sys
from io import StringIO
from contextlib import contextmanager

@contextmanager
def capture_output():
    """Capture stdout and stderr using a context manager."""
    # Create StringIO objects to capture output
    stdout, stderr = StringIO(), StringIO()
    
    # Save the current stdout/stderr
    old_stdout, old_stderr = sys.stdout, sys.stderr
    
    try:
        # Replace stdout/stderr with our StringIO objects
        sys.stdout, sys.stderr = stdout, stderr
        yield stdout, stderr
    finally:
        # Restore the original stdout/stderr
        sys.stdout, sys.stderr = old_stdout, old_stderr

## Create Training Corpus

We need to catalog CHEBI into chemical entities and classes, grouping the former by the latter

In [12]:
from oaklib import get_adapter

chebi = get_adapter("sqlite:obo:chebi")

In [13]:
from sqlalchemy.orm import aliased

session = chebi.session
from semsql.sqla.semsql import Statements, HasDbxrefStatement, RdfsSubclassOfStatement, RdfsLabelStatement, HasTextDefinitionStatement

smiles_tbl = aliased(Statements)

q = session.query(RdfsLabelStatement)
q = q.join(HasTextDefinitionStatement, HasTextDefinitionStatement.subject == RdfsLabelStatement.subject, isouter=True)
q = q.join(RdfsSubclassOfStatement, RdfsSubclassOfStatement.subject == RdfsLabelStatement.subject)
q = q.join(smiles_tbl, smiles_tbl.subject == RdfsLabelStatement.subject, isouter=True)
q = q.filter(smiles_tbl.predicate == "obo:chebi/smiles")


In [14]:
import pandas as pd
from sqlalchemy.orm import aliased
from sqlalchemy import and_

# Alias tables for clarity
smiles_tbl = aliased(Statements, name='smiles')
definition_tbl = aliased(HasTextDefinitionStatement, name='definition')
subclass_tbl = aliased(RdfsSubclassOfStatement, name='subclass')
label_tbl = aliased(RdfsLabelStatement, name='label')

# Build the query
q = (session.query(
        label_tbl.subject,
        label_tbl.value.label('label'),
        definition_tbl.value.label('definition'),
        subclass_tbl.object.label('parent_class'),
        smiles_tbl.value.label('smiles')
    )
    .select_from(label_tbl)
    # Left joins to preserve rows even if some data is missing
    .join(
        definition_tbl,
        definition_tbl.subject == label_tbl.subject,
        isouter=True
    )
    .join(
        subclass_tbl,
        subclass_tbl.subject == label_tbl.subject,
        isouter=False
    )
    .join(
        smiles_tbl,
        and_(
            smiles_tbl.subject == label_tbl.subject,
            smiles_tbl.predicate == "obo:chebi/smiles"
        ),
        isouter=True
    ))

# Execute query
results = q.all()
df = pd.DataFrame([{
    'id': r.subject,
    'label': r.label,
    'definition': r.definition,
    'parent': r.parent_class,
    'smiles': r.smiles
} for r in results])

df

,id,label,definition,parent,smiles
0,CHEBI:10,(+)-Atherospermoline,None,CHEBI:133004,COc1cc2CCN(C)[C@H]3Cc4ccc(Oc5cc(C[C@@H]6N(C)CC...
1,CHEBI:100,(-)-medicarpin,The (-)-enantiomer of medicarpin.,CHEBI:16114,[H][C@@]12COc3cc(O)ccc3[C@]1([H])Oc1cc(OC)ccc21
2,CHEBI:100,(-)-medicarpin,The (-)-enantiomer of medicarpin.,_:riog00000054,[H][C@@]12COc3cc(O)ccc3[C@]1([H])Oc1cc(OC)ccc21
3,CHEBI:100,(-)-medicarpin,The (-)-enantiomer of medicarpin.,_:riog00000055,[H][C@@]12COc3cc(O)ccc3[C@]1([H])Oc1cc(OC)ccc21
4,CHEBI:10000,Vismione D,None,CHEBI:46955,CC(C)=CCC\C(C)=C\COc1cc(O)c2c(O)c3C(=O)CC(C)(O...
...,...,...,...,...,...
371849,CHEBI:99997,"N-[(2S,4aS,12aS)-2-[2-(cyclohexylmethylamino)-...",None,CHEBI:36586,CN1[C@H]2CC[C@H](O[C@@H]2COC3=C(C1=O)C=C(C=C3)...
371850,CHEBI:99998,"N-[[(3S,9S,10R)-16-(dimethylamino)-12-[(2S)-1-...",None,CHEBI:24995,C[C@H]1CCCCO[C@@H]([C@@H](CN(C(=O)C2=C(O1)C=CC...
371851,CHEBI:99998,"N-[[(3S,9S,10R)-16-(dimethylamino)-12-[(2S)-1-...",None,CHEBI:52898,C[C@H]1CCCCO[C@@H]([C@@H](CN(C(=O)C2=C(O1)C=CC...
371852,CHEBI:99999,"N-[(5S,6S,9S)-5-methoxy-3,6,9-trimethyl-2-oxo-...",None,CHEBI:24995,C[C@H]1CN[C@H](COC2=C(C=CC(=C2)NC(=O)C3=NC4=CC...


In [15]:
df_collapsed = df.groupby('id').agg({
    'label': 'first',  # Take first label since it should be same
    'definition': 'first',  # Take first definition since it should be same
    'parent': lambda x: list(x.dropna().unique()),  # Collect all unique parents into a list
    'smiles': 'first'  # Take first SMILES since it should be same
}).reset_index()
df_collapsed

,id,label,definition,parent,smiles
0,CHEBI:10,(+)-Atherospermoline,None,[CHEBI:133004],COc1cc2CCN(C)[C@H]3Cc4ccc(Oc5cc(C[C@@H]6N(C)CC...
1,CHEBI:100,(-)-medicarpin,The (-)-enantiomer of medicarpin.,"[CHEBI:16114, _:riog00000054, _:riog00000055]",[H][C@@]12COc3cc(O)ccc3[C@]1([H])Oc1cc(OC)ccc21
2,CHEBI:10000,Vismione D,None,[CHEBI:46955],CC(C)=CCC\C(C)=C\COc1cc(O)c2c(O)c3C(=O)CC(C)(O...
3,CHEBI:100000,"(2S,3S,4R)-3-[4-(3-cyclopentylprop-1-ynyl)phen...",None,"[CHEBI:22712, CHEBI:36820, CHEBI:38777]",COCC(=O)N1[C@H]([C@H]([C@H]1C#N)C2=CC=C(C=C2)C...
4,CHEBI:100001,"N-[(2R,3S,6R)-2-(hydroxymethyl)-6-[2-[[oxo-[4-...",None,[CHEBI:20857],C1C[C@@H]([C@@H](O[C@H]1CCNC(=O)NC2=CC=C(C=C2)...
...,...,...,...,...,...
200903,CHEBI:99995,"2-[(2S,4aS,12aS)-5-methyl-6-oxo-8-[(1-oxo-2-ph...",None,[CHEBI:22160],CN1[C@H]2CC[C@H](O[C@@H]2COC3=C(C1=O)C=C(C=C3)...
200904,CHEBI:99996,"N-[(1S,3S,4aR,9aS)-3-[2-[(2,5-difluorophenyl)m...",None,[CHEBI:74927],C1[C@H](O[C@H]([C@@H]2[C@H]1C3=C(O2)C=CC(=C3)N...
200905,CHEBI:99997,"N-[(2S,4aS,12aS)-2-[2-(cyclohexylmethylamino)-...",None,"[CHEBI:17792, CHEBI:36586]",CN1[C@H]2CC[C@H](O[C@@H]2COC3=C(C1=O)C=C(C=C3)...
200906,CHEBI:99998,"N-[[(3S,9S,10R)-16-(dimethylamino)-12-[(2S)-1-...",None,"[CHEBI:24995, CHEBI:52898]",C[C@H]1CCCCO[C@@H]([C@@H](CN(C(=O)C2=C(O1)C=CC...


In [16]:
df = df_collapsed

In [17]:
counts = pd.crosstab(
    df['definition'].isna(),
    df['smiles'].isna(),
    margins=True
)
counts

smiles,False,True,All
definition,,,
False,46372,7708,54080
True,140627,6201,146828
All,186999,13909,200908


In [18]:
parent_to_children = {}
child_to_parents = {}
for _, row in df.iterrows():
    for parent in row['parent']:
        if parent not in parent_to_children:
            parent_to_children[parent] = []
        parent_to_children[parent].append(row['id'])
        if row['id'] not in child_to_parents:
            child_to_parents[row['id']] = []
        child_to_parents[row['id']].append(parent)

In [19]:
has_smiles = set(df[df['smiles'].notna()]['id'])
has_definition = set(df[df['definition'].notna()]['id'])


In [20]:
# Find parents meeting our criteria:
# 1. Parent doesn't have SMILES
# 2. All children have SMILES
# 3. Has at least one child
matching_parents = [
    parent for parent, children in parent_to_children.items()
    if (parent not in has_smiles and  # parent lacks SMILES
        parent in has_definition and  # parent has a definition
        len(children) > 0 and  # has children
        all(child in has_smiles for child in children))  # all children have SMILES
]

# Get the results in a nice dataframe
results = pd.DataFrame({
    'parent': matching_parents,
    'num_children': [len(parent_to_children[p]) for p in matching_parents],
    'children': [parent_to_children[p] for p in matching_parents]
})
results

,parent,num_children,children
0,CHEBI:133004,35,"[CHEBI:10, CHEBI:11, CHEBI:132893, CHEBI:13289..."
1,CHEBI:74927,1067,"[CHEBI:100017, CHEBI:100022, CHEBI:100028, CHE..."
2,CHEBI:137443,3,"[CHEBI:10002, CHEBI:173072, CHEBI:6133]"
3,CHEBI:73537,6,"[CHEBI:100147, CHEBI:101853, CHEBI:157175, CHE..."
4,CHEBI:27288,23,"[CHEBI:10016, CHEBI:104242, CHEBI:125454, CHEB..."
...,...,...,...
2126,CHEBI:39207,3,"[CHEBI:9230, CHEBI:9231, CHEBI:9232]"
2127,CHEBI:231697,1,[CHEBI:94998]
2128,CHEBI:39266,1,[CHEBI:9506]
2129,CHEBI:39267,1,[CHEBI:9506]


In [21]:
structures = {}
for _, row in df.iterrows():
    if row['id'] in has_smiles:
        structures[row['id']] = ChemicalStructure(name=row['label'], smiles=row['smiles'])


In [22]:
# Create all classes, indexed by id
classes = {}
for _, row in df.iterrows():
    if row['id'] not in matching_parents:
        continue
    cls = ChemicalClass(
        id = row['id'],
        name=row['label'],
        definition=row['definition'],
        instances=[structures[child] for child in parent_to_children[row['id']] if child in structures]
    )
    classes[cls.id] = cls

In [23]:
for cls in list(classes.values())[0:2]:
    print(cls)

id='CHEBI:11791' name='3-deoxy-D-manno-octulosonic acid' definition='An eight-membered ketoaldonic acid having D-manno configuration' instances=[ChemicalStructure(name='keto-3-deoxy-D-manno-octulosonic acid', smiles='OC[C@@H](O)[C@@H](O)[C@H](O)[C@H](O)CC(=O)C(O)=O'), ChemicalStructure(name='3-deoxy-alpha-D-manno-oct-2-ulopyranosonic acid', smiles='[H][C@@]1(O[C@](O)(C[C@@H](O)[C@H]1O)C(O)=O)[C@H](O)CO')] negative_instances=None
id='CHEBI:12164' name='5-phosphoribosyl diphosphate' definition='A  ribose diphosphate carrying an additional phosphate group at position 5.' instances=[ChemicalStructure(name='5-O-phosphono-D-ribofuranosyl diphosphate', smiles='O[C@H]1[C@@H](O)C(O[C@@H]1COP(O)(O)=O)OP(O)(=O)OP(O)(O)=O')] negative_instances=None


In [24]:
filtered_classes = [cls for cls in classes.values() if len(cls.instances) > 5]
len(filtered_classes)

615

In [25]:
# add negative instances
for cls in filtered_classes:
    negative_instances = []
    for p in child_to_parents.get(cls.id, []):
        for c2 in parent_to_children.get(p, []):
            if c2 == cls.id:
                continue
            if c2 in classes:
                c2_cls = classes[c2]
                for inst in c2_cls.instances:
                    negative_instances.append(inst)
    for i in list(structures.values())[:20]:
        negative_instances.append(i)
    cls.negative_instances = []
    pos_smiles = {i.smiles for i in cls.instances}
    for i in negative_instances:
        if i.smiles not in pos_smiles:
            cls.negative_instances.append(i)
    
    

In [26]:
lens = [len(cls.negative_instances) for cls in filtered_classes]
max(lens), min(lens)

(2732, 20)

## Utils

In [103]:
import pandas as pd
import numpy as np

def calculate_classification_metrics(tp, tn, fp, fn):
    """
    Calculate classification metrics from confusion matrix values.
    
    Parameters:
    tp (int/float): True positives
    tn (int/float): True negatives
    fp (int/float): False positives
    fn (int/float): False negatives
    
    Returns:
    dict: Dictionary containing various classification metrics
    """
    # Prevent division by zero by adding small epsilon
    eps = 1e-7
    
    # Basic counts
    total = tp + tn + fp + fn
    positives = tp + fp
    negatives = tn + fn
    actual_positives = tp + fn
    actual_negatives = tn + fp
    
    # Core metrics
    accuracy = (tp + tn) / (total + eps)
    precision = tp / (tp + fp + eps)
    recall = tp / (tp + fn + eps)
    specificity = tn / (tn + fp + eps)
    f1_score = 2 * (precision * recall) / (precision + recall + eps)
    
    # Additional metrics
    false_positive_rate = fp / (fp + tn + eps)
    false_negative_rate = fn / (fn + tp + eps)
    negative_predictive_value = tn / (tn + fn + eps)
    matthews_correlation_coef = ((tp * tn) - (fp * fn)) / np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn) + eps)
    balanced_accuracy = (recall + specificity) / 2
    
    return {
        # Counts
        'total_samples': total,
        'true_positives': tp,
        'true_negatives': tn,
        'false_positives': fp,
        'false_negatives': fn,
        
        # Core metrics
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'specificity': specificity,
        
        # Additional metrics
        'false_positive_rate': false_positive_rate,
        'false_negative_rate': false_negative_rate,
        'negative_predictive_value': negative_predictive_value,
        'matthews_correlation_coef': matthews_correlation_coef,
        'balanced_accuracy': balanced_accuracy
    }

# Function to work with your DataFrame
import pandas as pd

def calculate_metrics_pandas(data):
    """
    Calculate classification metrics using pandas operations.
    Works with both Series and DataFrame inputs containing:
    num_true_positives, num_true_negatives, num_false_positives, num_false_negatives
    """
    # If input is a Series, use the values directly
    if isinstance(data, pd.Series):
        tp = data['num_true_positives']
        tn = data['num_true_negatives']
        fp = data['num_false_positives']
        fn = data['num_false_negatives']
    # If input is a DataFrame, use iloc[0] to get the first row
    else:
        tp = data['num_true_positives'].iloc[0]
        tn = data['num_true_negatives'].iloc[0]
        fp = data['num_false_positives'].iloc[0]
        fn = data['num_false_negatives'].iloc[0]
    
    metrics = pd.Series({
        # Basic counts
        'total': tp + tn + fp + fn,
        'positives': tp + fp,
        'negatives': tn + fn,
        'actual_positives': tp + fn,
        'actual_negatives': tn + fp,
        
        # Core metrics
        'accuracy': (tp + tn) / (tp + tn + fp + fn),
        'precision': tp / (tp + fp),
        'recall': tp / (tp + fn),
        'specificity': tn / (tn + fp),
        
        # Additional metrics
        'f1_score': 2 * tp / (2 * tp + fp + fn),
        'false_positive_rate': fp / (tn + fp),
        'negative_predictive_value': tn / (tn + fn),
    })
    
    metrics['balanced_accuracy'] = (metrics['recall'] + metrics['specificity']) / 2
    
    return metrics.round(4)

## Main workflow

The main workflow is a cycle between

1. Generating code
2. Running the code on positive and negative examples
3. Go to 1 until `N` iterations or sufficient accuracy is received

In [158]:
from typing import Optional

class Config(BaseModel):
    """Experimental setup"""
    llm_model_name: str = "gpt-4o"
    accuracy_threshold: float = 0.5
    max_attempts: int = 3
    max_negative: int = 20

OUTCOME = Tuple[str, Optional[str]]
class Result(BaseModel):
    """Result of running workflow on a chemical class"""
    chemical_class: ChemicalClass
    config: Optional[Config] = None
    code: str
    true_positives: Optional[List[OUTCOME]] = None
    false_positives: Optional[List[OUTCOME]] = None
    true_negatives: Optional[List[OUTCOME]] = None
    false_negatives: Optional[List[OUTCOME]] = None
    attempt: int = 0
    success: bool = True
    best: bool = False
    error: Optional[str] = None
    stdout: Optional[str] = None
    
    num_true_positives: Optional[int] = None
    num_false_positives: Optional[int] = None
    num_true_negatives: Optional[int] = None
    num_false_negatives: Optional[int] = None
    
    precision: Optional[float] = None
    recall: Optional[float] = None
    f1: Optional[float] = None
    
    def calculate(self):
        """Calculate derived statistics"""
        self.num_true_positives = len(self.true_positives or [])
        self.num_false_positives = len(self.false_positives or [])
        self.num_true_negatives = len(self.true_negatives or [])
        self.num_false_negatives = len(self.false_negatives or [])
        if self.num_true_positives + self.num_false_positives:
            self.precision = self.num_true_positives / (self.num_true_positives + self.num_false_positives)
        else:
            self.precision = 0.0
        if self.num_true_positives + self.num_false_negatives:
            self.recall = self.num_true_positives / (self.num_true_positives + self.num_false_negatives)
        else:
            self.recall = 0
        if self.precision and self.recall:
            self.f1 = 2 * (self.precision * self.recall) / (self.precision + self.recall)
        else:
            self.f1 = 0
        

In [168]:
from llm import get_key
from typing import Iterator
from pathlib import Path

test_dir = Path("tmp")
def generate_and_test_classifier(cls: ChemicalClass, attempt=0, err=None, prog=None, config=None, suppress_llm=False) -> Iterator[Result]:
    """
    Main workflow
    
    :param cls: target chemical class for which is write a program
    :param attempt: counts which attempt this is
    :param err: error from previous iteration
    :param prog: program from previous iteration
    :param config: setup
    :return: 
    """
    if config is None:
        config = Config()
    next_attempt = attempt + 1
    if next_attempt > config.max_attempts:
        print(f"FAILED: {cls.name} err={err[0:40]}")
        return
    safe = safe_name(cls.name)
    func_name = f"is_{safe}"
    if suppress_llm:
        code_str = prog
    else:
        system_prompt = generate_system_prompt(validated_examples)
        main_prompt = generate_main_prompt(cls.name, cls.definition, cls.instances, err=err, prog=prog)
        # print(main_prompt)
        model = get_model(config.llm_model_name)
        if model.needs_key:
            model.key = get_key(None, model.needs_key, model.key_env_var)
        if "o1" in config.llm_model_name:
            response = model.prompt(f"SYSTEM PROMPT: {system_prompt} MAIN PROMPT: {main_prompt}")
        else:
            response = model.prompt(main_prompt, system=system_prompt)
        code_str = response.text()
        if not code_str:
            print(f"No code returned for {cls.name} // {response}")
        if "```" in code_str:  # Remove code block markdown
            code_str = code_str.split("```")[1].strip()
            if code_str.startswith("python"):
                code_str = code_str[6:]
            code_str = code_str.strip()
        # print(code_str)
    
    positive_instances = cls.instances
    negative_instances = cls.negative_instances[0:config.max_negative]
    #negative_instances = []
    smiles_to_cls = {instance.smiles: True for instance in positive_instances}
    #for s in structures.values():
    #    if s.smiles not in smiles_to_cls:
    #        negative_instances.append(s)
    #    if len(negative_instances) >= len(positive_instances):
    #        break
    for instance in negative_instances:
        smiles_to_cls[instance.smiles] = False
    try:
        with capture_output() as (stdout, stderr):
            results = run_code(code_str, func_name, [instance.smiles for instance in positive_instances + negative_instances])
    except Exception as e:
        yield Result(
            chemical_class=cls,
            config=config,
            code=code_str,
            attempt=attempt,
            success=False,
            error=str(e),
        )
        msg = "Attempt failed: " + str(e)
        if not suppress_llm:
            yield from generate_and_test_classifier(cls, attempt=next_attempt, config=config, err=msg, prog=code_str)
        return
    true_positives = [(smiles, reason) for smiles, is_cls, reason in results if is_cls and smiles_to_cls[smiles]]
    true_negatives = [(smiles, reason) for smiles, is_cls, reason in results if not is_cls and not smiles_to_cls[smiles]]
    false_positives = [(smiles, reason) for smiles, is_cls, reason in results if is_cls and not smiles_to_cls[smiles]]
    false_negatives = [(smiles, reason) for smiles, is_cls, reason in results if not is_cls and smiles_to_cls[smiles]]
    result = Result(
        chemical_class=cls,
        config=config,
        code=code_str,
        true_positives=true_positives,
        false_positives=false_positives,
        true_negatives=true_negatives,
        false_negatives=false_negatives,
        attempt=attempt,
        stdout=stdout.getvalue(),
        error=stderr.getvalue(),
        success=True,
    )
    result.calculate()
    yield result
    if suppress_llm:
        return
    if result.f1 is None or result.f1 < config.accuracy_threshold and not suppress_llm:
        msg = f"\nAttempt failed: F1 score of {result.f1} is too low"
        msg += "\nTrue positives: " + str(true_positives)
        msg += "\nFalse positives: " + str(false_positives)
        msg += "\nFalse negatives: " + str(false_negatives)
        yield from generate_and_test_classifier(cls, config=config, attempt=next_attempt, err=msg, prog=code_str)
        

In [86]:
from copy import copy
import random


def split_to_training_test(classes: List[ChemicalClass], proportion_test=0.2, n: int = 9999, start: int = 0) -> Tuple[List[ChemicalClass], List[ChemicalClass]]:
    test_set = []
    training_set = []
    for c in classes[start:n+start]:
        test_c = copy(c)
        train_c = copy(c)
        positive_examples = copy(c.instances)
        negative_examples = copy(c.negative_instances)
        random.shuffle(positive_examples)
        random.shuffle(negative_examples)
        i_positive = int(len(positive_examples) * proportion_test)
        i_negative = int(len(negative_instances) * proportion_test)
        test_c.instances = positive_examples[:i_positive]
        test_c.negative_instances = negative_examples[:i_negative]
        train_c.instances = positive_examples[i_positive:]
        train_c.negative_instances = negative_examples[i_negative:]
        test_set.append(test_c)
        training_set.append(train_c)
    return training_set, test_set
        
        
        

In [30]:
a, b = split_to_training_test(filtered_classes, n=3)

In [31]:
a[0].instances

[ChemicalStructure(name='all-trans-3,4-didehydroretinoic acid', smiles='C1(C)(C)CC=CC(=C1\\C=C\\C(=C\\C=C\\C(=C\\C(=O)O)\\C)\\C)C'),
 ChemicalStructure(name='all-trans-retinal', smiles='[H]C(=O)\\C=C(/C)\\C=C\\C=C(/C)\\C=C\\C1=C(C)CCCC1(C)C'),
 ChemicalStructure(name='all-trans-retinol', smiles='C\\C(=C/CO)\\C=C\\C=C(/C)\\C=C\\C1=C(C)CCCC1(C)C'),
 ChemicalStructure(name='all-trans-retinoic acid', smiles='CC(\\C=C\\C1=C(C)CCCC1(C)C)=C/C=C/C(C)=C/C(O)=O'),
 ChemicalStructure(name='all-trans-3,4-didehydroretinol', smiles='C1(C)(C)C(\\C=C\\C(=C\\C=C\\C(=C\\CO)\\C)\\C)=C(C)C=CC1')]

In [32]:
b[0].instances

[ChemicalStructure(name='all-trans-retinyl ester', smiles='CC(\\C=C\\C=C(C)\\C=C\\C1=C(C)CCCC1(C)C)=C/COC([*])=O')]

## Run an individual experiment

In [125]:
# claude-sonnet seems best so far
config = Config(llm_model_name="lbl/claude-sonnet", max_attempts=5, accuracy_threshold=0.95)

In [126]:
len(filtered_classes)

615

In [127]:
training_set, test_set = split_to_training_test(filtered_classes, n=100, start=400)
len(training_set)


100

In [129]:
results = []
for test_cls in training_set:
    print("##", test_cls.name)
    for result in generate_and_test_classifier(test_cls, config=config):
        print(result.attempt, result.num_true_positives, result.num_true_negatives, result.num_false_positives, result.f1)
        results.append(result)
        result.calculate()

## cyclic tetrapyrrole anion
0 None None None None
1 None None None None
2 None None None None
3 None None None None
4 None None None None
FAILED: cyclic tetrapyrrole anion err=Attempt failed: (unicode error) 'unicodeescape' codec can't decode bytes in position 72-73: malformed \N character escape (<string>, line 1)
## alpha-amino-acid cation residue
15 1
0 15 11 0 0.967741935483871
15 1
## acyl-CoA oxoanion
60 0
0 60 20 0 1.0
60 0
## straight-chain saturated fatty acid anion
18 2
0 18 20 0 0.9473684210526316
18 2
0 20
1 0 20 0 0
0 20
19 1
2 19 20 0 0.9743589743589743
19 1
## branched-chain saturated fatty acid anion
13 0
0 13 16 4 0.8666666666666666
13 0
13 0
1 13 16 4 0.8666666666666666
13 0
13 0
2 13 16 4 0.8666666666666666
13 0
13 0
3 13 16 4 0.8666666666666666
13 0
13 0
4 13 12 8 0.7647058823529412
13 0
FAILED: branched-chain saturated fatty acid anion err=
Attempt failed: F1 score of 0.7647058823529412 is too low
True positives: [('CC(C)CCCC(C)CCCC(C)CCCC(C)CC([O-])=O', 'Branched

KeyboardInterrupt: 

In [134]:
print(len(results))

203


In [177]:
def calc_eval_results(results, min_f1=0):
    eval_results = []
    for result in results:
        if result.f1 < min_f1:
            continue
        # print(result.f1)
        train_cls = result.chemical_class
        code = result.code
        [test_cls] = [c for c in test_set if c.id == train_cls.id]
        for eval_result in generate_and_test_classifier(test_cls, suppress_llm=True, prog=code, config=config):
            eval_results.append(eval_result)
            eval_result.calculate()
            # print(eval_result.f1)
    return pd.DataFrame([r.model_dump() for r in eval_results])



    

In [191]:
type(results)

list

False

In [187]:
def calculate_best():
    best_by_cls = {}
    for r in results:
        cid = r.chemical_class.id
        if r.f1 and (cid not in best_by_cls or r.f1 > best_by_cls[cid]):
            best_by_cls[cid] = r.f1
    for r in results:
        r.best = False
        cid = r.chemical_class.id
        if cid in best_by_cls and best_by_cls[cid] == r.f1:
            r.best = True
            
calculate_best()

In [193]:
eval_df = calc_eval_results([r for r in results if r.best]) 

In [194]:
results_dir = Path("latest")
results_dir.mkdir(parents=True, exist_ok=True)
with open(results_dir / "results.json", "w") as f:
    import json
    results_objs = [r.model_dump() for r in results]
    f.write(json.dumps(results_objs, indent=2))

In [190]:
type(eval_results)

pandas.core.frame.DataFrame

In [181]:
def results_as_df(results):
    rows = []
    for r in results:
        r.calculate()
        row = r.model_dump()
        rows.append(row)
    return pd.DataFrame(rows)
        

In [182]:
#eval_df = results_as_df(eval_results)

AttributeError: 'str' object has no attribute 'calculate'

In [195]:
eval_df.to_csv( results_dir / "eval_results.csv")

In [196]:
def df_stats(df):
    return calculate_metrics_pandas(df.aggregate({"num_true_positives": "sum", "num_true_negatives": "sum", "num_false_positives": "sum",  "num_false_negatives": "sum"}))

In [197]:
eval_df.aggregate({"num_true_positives": "sum", "num_true_negatives": "sum", "num_false_positives": "sum",  "num_false_negatives": "sum"})

num_true_positives     254
num_true_negatives     468
num_false_positives     36
num_false_negatives    336
dtype: int64

In [200]:
df_stats(calc_eval_results(results, min_f1=1.0))

total                        159.0000
positives                     47.0000
negatives                    112.0000
actual_positives              51.0000
actual_negatives             108.0000
accuracy                       0.9748
precision                      1.0000
recall                         0.9216
specificity                    1.0000
f1_score                       0.9592
false_positive_rate            0.0000
negative_predictive_value      0.9643
balanced_accuracy              0.9608
dtype: float64

In [110]:
df_stats(eval_df.query("f1 == 1.0"))

total                        171.0
positives                     54.0
negatives                    117.0
actual_positives              54.0
actual_negatives             117.0
accuracy                       1.0
precision                      1.0
recall                         1.0
specificity                    1.0
f1_score                       1.0
false_positive_rate            0.0
negative_predictive_value      1.0
balanced_accuracy              1.0
dtype: float64

In [98]:
results_df = results_as_df(results)

0 8
0 8
2 6
25 1
103 1
0 8
0 8
0 8
8 0
0 5
0 5
0 5
0 52
37 15
3 3
0 6
0 6
0 6
37 2
0 20
0 20
0 20
0 20
0 102
64 38
0 102
0 5
4 1
45 0
0 41
0 41
0 41
0 6
0 6
0 6
0 6
1 26
27 0
0 6
0 6
0 6
0 6
10 0
0 24
0 24
0 24
6 18
0 59
0 59
0 59
0 11
0 11
0 11
0 11
18 14
32 0
8 0
0 12
0 12
0 12
0 12
0 6
6 0
0 18
0 18
0 18
0 18
0 7
0 7
0 7
0 7
13 1
0 5
5 0
21 2
8 1
0 10
10 0
15 71
0 86
0 86
0 5
0 5
0 5
0 5
24 0
8 0
0 13
0 13
0 13
0 13
0 10
0 10
0 10
0 10
2 120
0 122
0 122
0 122
0 854


In [69]:
results_df.query('best == True')


,chemical_class,config,code,true_positives,false_positives,true_negatives,false_negatives,attempt,success,best,error,stdout,num_true_positives,num_false_positives,num_true_negatives,num_false_negatives,precision,recall,f1
0,"{'id': 'CHEBI:12777', 'name': 'vitamin A', 'de...","{'llm_model_name': 'lbl/claude-sonnet', 'accur...",from rdkit import Chem\nfrom rdkit.Chem import...,[(CC(\C=C\C1=C(C)CCCC1(C)C)=C/C=C/C(C)=C/C(O)=...,[],[(C[C@H]1CN[C@H](COC2=C(C=CC(=C2)NS(=O)(=O)C3=...,[(CC(\C=C\C=C(C)\C=C\C1=C(C)CCCC1(C)C)=C/COC([...,0,True,True,,,4,0,14,1,1.000000,0.800000,0.888889
10,"{'id': 'CHEBI:131619', 'name': 'C27-steroid', ...","{'llm_model_name': 'lbl/claude-sonnet', 'accur...",from rdkit import Chem\nfrom rdkit.Chem import...,[([H][C@@]1(CC[C@@]2([H])[C@]3([H])CC=C4C[C@@H...,[(C[C@H](CC[C@@H]1OC1(C)C)[C@H]1CC[C@H]2[C@@H]...,[(CCCC(=O)NC1=CC2=C(C=C1)OC[C@H](N(C[C@@H]([C@...,[([H]C(=O)C1C2CC(O)CCC2(C)C2CCC3(C)C(CCC3([H])...,1,True,True,,,18,3,17,2,0.857143,0.900000,0.878049
11,"{'id': 'CHEBI:131620', 'name': 'C24-steroid', ...","{'llm_model_name': 'lbl/claude-sonnet', 'accur...",from rdkit import Chem\nfrom rdkit.Chem import...,[([H][C@@]12C[C@H](O)CC[C@]1(C)[C@@]1([H])CC[C...,[],[(C[C@@H]1CN[C@H](COC2=C(C=C(C=C2)NS(=O)(=O)C3...,[],0,True,True,,,12,0,11,0,1.000000,1.000000,1.000000
13,"{'id': 'CHEBI:131621', 'name': 'C19-steroid', ...","{'llm_model_name': 'lbl/claude-sonnet', 'accur...",from rdkit import Chem\nfrom rdkit.Chem import...,[(C1[C@@]2([C@]3(CC[C@]4([C@]([C@@]3(CC=C2CC(C...,[(C[C@]12CC[C@H]3[C@@H](CC[C@H]4CCCC[C@]34C)[C...,[(C[C@@H]1CN[C@H](COC2=C(C=C(C=C2)NS(=O)(=O)C3...,[(C1=C2C(CC[C@]3([C@@]4(CC[C@@H]([C@]4(CC[C@@]...,1,True,True,,,5,1,19,1,0.833333,0.833333,0.833333
21,"{'id': 'CHEBI:131858', 'name': 'HETE anion', '...","{'llm_model_name': 'lbl/claude-sonnet', 'accur...",from rdkit import Chem\nfrom rdkit.Chem import...,[(C(=C\[C@H](C/C=C\CCCCC)O)/C=C\C/C=C\CCCC(=O)...,[(C(=O)([O-])CCCC(/C=C/C=C\C/C=C\C/C=C\CCCCCO)...,[(C(\CC)=C\CC1C(C/C=C\C/C=C\C/C=C\CCCC(=O)[O-]...,[],3,True,True,,,11,1,19,0,0.916667,1.000000,0.956522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,"{'id': 'CHEBI:197387', 'name': 'hexadecanol', ...","{'llm_model_name': 'lbl/claude-sonnet', 'accur...",from rdkit import Chem\nfrom rdkit.Chem import...,"[(C(CCCCCCCCCCCCCCC)O, Hexadecan-1-ol), (CCCCC...",[],"[(CCCCCCCCCCCCCCCC(O)CCCCCCCCC, Contains 25 ca...",[],0,True,True,,,7,0,20,0,1.000000,1.000000,1.000000
291,"{'id': 'CHEBI:197399', 'name': 'heptadecanol',...","{'llm_model_name': 'lbl/claude-sonnet', 'accur...",from rdkit import Chem\nfrom rdkit.Chem import...,"[(CCCCCCCCC(O)CCCCCCCC, Matches heptadecanol p...",[],"[(CCCCCCCCCCCCCCCCCCCCCCC(O)CC, Contains 25 ca...",[],2,True,True,,,8,0,20,0,1.000000,1.000000,1.000000
292,"{'id': 'CHEBI:197457', 'name': 'octadecanol', ...","{'llm_model_name': 'lbl/claude-sonnet', 'accur...",from rdkit import Chem\nfrom rdkit.Chem import...,"[(CCCCCCCCCC(O)CCCCCCCC, Octadecan-10-ol), (CC...",[],"[(CCCCCCCCCCCCCCCC(O)CCCCC, Wrong molecular fo...",[],0,True,True,,,8,0,20,0,1.000000,1.000000,1.000000
293,"{'id': 'CHEBI:197468', 'name': 'nonadecanol', ...","{'llm_model_name': 'lbl/claude-sonnet', 'accur...",from rdkit import Chem\nfrom rdkit.Chem import...,"[(CCCCCCCCCC(O)CCCCCCCCC, Nonadecanol with OH ...",[],"[(FC(Cl)(Cl)Cl, Molecular formula C1H0O0 does ...",[],0,True,True,,,8,0,20,0,1.000000,1.000000,1.000000


In [70]:
results_df.query('best == True').aggregate({"precision": "mean", "recall": "mean", "f1": "mean"})


precision    0.958490
recall       0.899527
f1           0.909853
dtype: float64

In [71]:
results_df.query('best == True and precision == 1.0')

,chemical_class,config,code,true_positives,false_positives,true_negatives,false_negatives,attempt,success,best,error,stdout,num_true_positives,num_false_positives,num_true_negatives,num_false_negatives,precision,recall,f1
0,"{'id': 'CHEBI:12777', 'name': 'vitamin A', 'de...","{'llm_model_name': 'lbl/claude-sonnet', 'accur...",from rdkit import Chem\nfrom rdkit.Chem import...,[(CC(\C=C\C1=C(C)CCCC1(C)C)=C/C=C/C(C)=C/C(O)=...,[],[(C[C@H]1CN[C@H](COC2=C(C=CC(=C2)NS(=O)(=O)C3=...,[(CC(\C=C\C=C(C)\C=C\C1=C(C)CCCC1(C)C)=C/COC([...,0,True,True,,,4,0,14,1,1.0,0.800,0.888889
11,"{'id': 'CHEBI:131620', 'name': 'C24-steroid', ...","{'llm_model_name': 'lbl/claude-sonnet', 'accur...",from rdkit import Chem\nfrom rdkit.Chem import...,[([H][C@@]12C[C@H](O)CC[C@]1(C)[C@@]1([H])CC[C...,[],[(C[C@@H]1CN[C@H](COC2=C(C=C(C=C2)NS(=O)(=O)C3...,[],0,True,True,,,12,0,11,0,1.0,1.000,1.000000
23,"{'id': 'CHEBI:131859', 'name': 'oxo-ETE anion'...","{'llm_model_name': 'lbl/claude-sonnet', 'accur...",from rdkit import Chem\nfrom rdkit.Chem import...,[(C(CCC\C=C/C=C/C(C/C=C\C/C=C\CCCCC)=O)(=O)[O-...,[],[(C([C@H](/C=C/C=C/C=C\[C@H](CCCC([O-])=O)O)O)...,[],1,True,True,,,5,0,20,0,1.0,1.000,1.000000
24,"{'id': 'CHEBI:131862', 'name': 'HPODE(1-)', 'd...","{'llm_model_name': 'lbl/claude-sonnet', 'accur...",from rdkit import Chem\nfrom rdkit.Chem import...,"[(C(=C\[C@H](/C=C\CCCCC)OO)\CCCCCCCC(=O)[O-], ...",[],[(C1=CC=C(C=C1)[C@@H]2[C@H](N([C@H]2C#N)S(=O)(...,[],0,True,True,,,8,0,20,0,1.0,1.000,1.000000
42,"{'id': 'CHEBI:131878', 'name': 'cholanic acid ...","{'llm_model_name': 'lbl/claude-sonnet', 'accur...",from rdkit import Chem\nfrom rdkit.Chem import...,[([H][C@@]12C[C@H](O)CC[C@]1(C)[C@@]1([H])C[C@...,[],[(C1=CC=C(C=C1)[C@@H]2[C@H](N([C@H]2C#N)S(=O)(...,[],1,True,True,,,12,0,11,0,1.0,1.000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,"{'id': 'CHEBI:197387', 'name': 'hexadecanol', ...","{'llm_model_name': 'lbl/claude-sonnet', 'accur...",from rdkit import Chem\nfrom rdkit.Chem import...,"[(C(CCCCCCCCCCCCCCC)O, Hexadecan-1-ol), (CCCCC...",[],"[(CCCCCCCCCCCCCCCC(O)CCCCCCCCC, Contains 25 ca...",[],0,True,True,,,7,0,20,0,1.0,1.000,1.000000
291,"{'id': 'CHEBI:197399', 'name': 'heptadecanol',...","{'llm_model_name': 'lbl/claude-sonnet', 'accur...",from rdkit import Chem\nfrom rdkit.Chem import...,"[(CCCCCCCCC(O)CCCCCCCC, Matches heptadecanol p...",[],"[(CCCCCCCCCCCCCCCCCCCCCCC(O)CC, Contains 25 ca...",[],2,True,True,,,8,0,20,0,1.0,1.000,1.000000
292,"{'id': 'CHEBI:197457', 'name': 'octadecanol', ...","{'llm_model_name': 'lbl/claude-sonnet', 'accur...",from rdkit import Chem\nfrom rdkit.Chem import...,"[(CCCCCCCCCC(O)CCCCCCCC, Octadecan-10-ol), (CC...",[],"[(CCCCCCCCCCCCCCCC(O)CCCCC, Wrong molecular fo...",[],0,True,True,,,8,0,20,0,1.0,1.000,1.000000
293,"{'id': 'CHEBI:197468', 'name': 'nonadecanol', ...","{'llm_model_name': 'lbl/claude-sonnet', 'accur...",from rdkit import Chem\nfrom rdkit.Chem import...,"[(CCCCCCCCCC(O)CCCCCCCCC, Nonadecanol with OH ...",[],"[(FC(Cl)(Cl)Cl, Molecular formula C1H0O0 does ...",[],0,True,True,,,8,0,20,0,1.0,1.000,1.000000


In [72]:
results_df.query('success == True')

,chemical_class,config,code,true_positives,false_positives,true_negatives,false_negatives,attempt,success,best,error,stdout,num_true_positives,num_false_positives,num_true_negatives,num_false_negatives,precision,recall,f1
0,"{'id': 'CHEBI:12777', 'name': 'vitamin A', 'de...","{'llm_model_name': 'lbl/claude-sonnet', 'accur...",from rdkit import Chem\nfrom rdkit.Chem import...,[(CC(\C=C\C1=C(C)CCCC1(C)C)=C/C=C/C(C)=C/C(O)=...,[],[(C[C@H]1CN[C@H](COC2=C(C=CC(=C2)NS(=O)(=O)C3=...,[(CC(\C=C\C=C(C)\C=C\C1=C(C)CCCC1(C)C)=C/COC([...,0,True,True,,,4,0,14,1,1.0,0.800,0.888889
1,"{'id': 'CHEBI:131437', 'name': 'pyrrolobenzodi...","{'llm_model_name': 'lbl/claude-sonnet', 'accur...",from rdkit import Chem\nfrom rdkit.Chem import...,[],[],[(CC1=C2C(=CC=C1)NC3=C2N=CN=C3N4CCC5(CC4)OCCO5...,[(O=C1N2[C@@H](O)CC[C@H]2C=NC3=C1C=C(OC)C(=C3)...,0,True,False,,,0,0,20,6,0.0,0.000,0.000000
2,"{'id': 'CHEBI:131437', 'name': 'pyrrolobenzodi...","{'llm_model_name': 'lbl/claude-sonnet', 'accur...",from rdkit import Chem\nfrom rdkit.Chem import...,[],[],[(CC1=C2C(=CC=C1)NC3=C2N=CN=C3N4CCC5(CC4)OCCO5...,[(O=C1N2[C@@H](O)CC[C@H]2C=NC3=C1C=C(OC)C(=C3)...,1,True,False,,,0,0,20,6,0.0,0.000,0.000000
3,"{'id': 'CHEBI:131437', 'name': 'pyrrolobenzodi...","{'llm_model_name': 'lbl/claude-sonnet', 'accur...",from rdkit import Chem\nfrom rdkit.Chem import...,[],[],[(CC1=C2C(=CC=C1)NC3=C2N=CN=C3N4CCC5(CC4)OCCO5...,[(O=C1N2[C@@H](O)CC[C@H]2C=NC3=C1C=C(OC)C(=C3)...,2,True,False,,,0,0,20,6,0.0,0.000,0.000000
4,"{'id': 'CHEBI:131437', 'name': 'pyrrolobenzodi...","{'llm_model_name': 'lbl/claude-sonnet', 'accur...",from rdkit import Chem\nfrom rdkit.Chem import...,[],[],[(CC1=C2C(=CC=C1)NC3=C2N=CN=C3N4CCC5(CC4)OCCO5...,[(O=C1N2[C@@H](O)CC[C@H]2C=NC3=C1C=C(OC)C(=C3)...,3,True,False,,,0,0,20,6,0.0,0.000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,"{'id': 'CHEBI:197399', 'name': 'heptadecanol',...","{'llm_model_name': 'lbl/claude-sonnet', 'accur...",from rdkit import Chem\nfrom rdkit.Chem import...,"[(CCCCCCCCC(O)CCCCCCCC, Matches heptadecanol p...",[],"[(CCCCCCCCCCCCCCCCCCCCCCC(O)CC, Contains 25 ca...",[],2,True,True,,,8,0,20,0,1.0,1.000,1.000000
292,"{'id': 'CHEBI:197457', 'name': 'octadecanol', ...","{'llm_model_name': 'lbl/claude-sonnet', 'accur...",from rdkit import Chem\nfrom rdkit.Chem import...,"[(CCCCCCCCCC(O)CCCCCCCC, Octadecan-10-ol), (CC...",[],"[(CCCCCCCCCCCCCCCC(O)CCCCC, Wrong molecular fo...",[],0,True,True,,,8,0,20,0,1.0,1.000,1.000000
293,"{'id': 'CHEBI:197468', 'name': 'nonadecanol', ...","{'llm_model_name': 'lbl/claude-sonnet', 'accur...",from rdkit import Chem\nfrom rdkit.Chem import...,"[(CCCCCCCCCC(O)CCCCCCCCC, Nonadecanol with OH ...",[],"[(FC(Cl)(Cl)Cl, Molecular formula C1H0O0 does ...",[],0,True,True,,,8,0,20,0,1.0,1.000,1.000000
295,"{'id': 'CHEBI:197480', 'name': 'icosanol', 'de...","{'llm_model_name': 'lbl/claude-sonnet', 'accur...",from rdkit import Chem\nfrom rdkit.Chem import...,"[(CCCCCCCCCCCC(O)CCCCCCCC, Icosan-12-ol), (CCC...",[],"[(ClC=C(Cl)Cl, Wrong molecular formula: C2HCl3...","[(CCCCCCCCCCCCCCCCCCC(C)O, Contains branching)]",1,True,True,,,7,0,20,1,1.0,0.875,0.933333


In [73]:
results_df.query('success == False')

,chemical_class,config,code,true_positives,false_positives,true_negatives,false_negatives,attempt,success,best,error,stdout,num_true_positives,num_false_positives,num_true_negatives,num_false_negatives,precision,recall,f1
9,"{'id': 'CHEBI:131619', 'name': 'C27-steroid', ...","{'llm_model_name': 'lbl/claude-sonnet', 'accur...",from rdkit import Chem\nfrom rdkit.Chem import...,None,None,None,None,0,False,False,cannot import name 'rdDecomposition' from 'rdk...,None,0,0,0,0,0.0,0.0,0.0
14,"{'id': 'CHEBI:131697', 'name': 'pyrimidotriazi...","{'llm_model_name': 'lbl/claude-sonnet', 'accur...",from rdkit import Chem\nfrom rdkit.Chem import...,None,None,None,None,0,False,False,No module named 'rdkit.Chem.rdDecomposition',None,0,0,0,0,0.0,0.0,0.0
18,"{'id': 'CHEBI:131858', 'name': 'HETE anion', '...","{'llm_model_name': 'lbl/claude-sonnet', 'accur...",from rdkit import Chem\nfrom rdkit.Chem import...,None,None,None,None,0,False,False,Python argument types in\n Mol.HasSubstruct...,None,0,0,0,0,0.0,0.0,0.0
26,"{'id': 'CHEBI:131867', 'name': 'hydroxydocosah...","{'llm_model_name': 'lbl/claude-sonnet', 'accur...",from rdkit import Chem\nfrom rdkit.Chem import...,None,None,None,None,1,False,False,module 'rdkit.Chem.rdMolDescriptors' has no at...,None,0,0,0,0,0.0,0.0,0.0
29,"{'id': 'CHEBI:131868', 'name': 'hydroperoxydoc...","{'llm_model_name': 'lbl/claude-sonnet', 'accur...",from rdkit import Chem\nfrom rdkit.Chem import...,None,None,None,None,0,False,False,module 'rdkit.Chem.rdMolDescriptors' has no at...,None,0,0,0,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,"{'id': 'CHEBI:19281', 'name': '2,2'-bithiophen...","{'llm_model_name': 'lbl/claude-sonnet', 'accur...",from rdkit import Chem\nfrom rdkit.Chem import...,None,None,None,None,0,False,False,No module named 'rdkit.Chem.rdDecomposition',None,0,0,0,0,0.0,0.0,0.0
280,"{'id': 'CHEBI:195620', 'name': 'tridecanol', '...","{'llm_model_name': 'lbl/claude-sonnet', 'accur...",from rdkit import Chem\nfrom rdkit.Chem import...,None,None,None,None,1,False,False,module 'rdkit.Chem.rdMolDescriptors' has no at...,None,0,0,0,0,0.0,0.0,0.0
283,"{'id': 'CHEBI:195629', 'name': 'pentadecanol',...","{'llm_model_name': 'lbl/claude-sonnet', 'accur...",from rdkit import Chem\nfrom rdkit.Chem import...,None,None,None,None,0,False,False,module 'rdkit.Chem.rdMolDescriptors' has no at...,None,0,0,0,0,0.0,0.0,0.0
290,"{'id': 'CHEBI:197399', 'name': 'heptadecanol',...","{'llm_model_name': 'lbl/claude-sonnet', 'accur...",from rdkit import Chem\nfrom rdkit.Chem import...,None,None,None,None,1,False,False,module 'rdkit.Chem.rdMolDescriptors' has no at...,None,0,0,0,0,0.0,0.0,0.0


In [74]:
slim_df = results_df.copy()
slim_df["code"] = ""

In [75]:
slim_df.to_csv(results_dir / "results.csv")

In [76]:
for r in results:
    cn = safe_name(r.chemical_class.name)
    prog_dir = results_dir / "programs"
    prog_dir.mkdir(exist_ok=True, parents=True)
    prog_path = f"{prog_dir / cn}.py"
    #print(prog_path)
    with open(prog_path, "w") as f:
        f.write(r.code)
        f.write(f"\n# Pr={r.precision}")
        f.write(f"\n# Recall={r.recall}")
    